In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset,TensorDataset
from autoencoder import Autoencoder
import torchvision
from model2 import classification_model
import copy
import partition
from pca import PCADigitReducer
from autoencoder import reduce_dimensions
from training import train,test, train_fashion,test_fashion
from federated_learning import distribute_global_model, federated_averaging
from model4 import MultilayerPerceptron
import cluster
from autoencoder2 import Autoencoder2

# Preprocessing

In [44]:
# Predefined stuff

n_epochs = 10
batch_size_train = 100
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10
num_clusters = 2

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [6]:
fashion_mnist_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.0,), (1.0,))  
])

fashion_mnist_train_loader = DataLoader(
    datasets.FashionMNIST('/files/', train=True, download=True, transform=fashion_mnist_transform),
    batch_size=batch_size_train, shuffle=True
)

fashion_mnist_test_loader = DataLoader(
    datasets.FashionMNIST('/files/', train=False, download=True, transform=fashion_mnist_transform),
    batch_size=batch_size_test, shuffle=True
)

In [7]:
train_loader_pca = copy.copy(fashion_mnist_train_loader)
test_loader_pca = copy.copy(fashion_mnist_test_loader)

train_loader_auto = copy.copy(fashion_mnist_train_loader)
test_loader_auto = copy.copy(fashion_mnist_test_loader)

In [8]:
class CustomTensorDataset(TensorDataset):
    def __init__(self, *tensors):
        super().__init__(*tensors)
        self.data = tensors[0]
        self.targets = tensors[1] 

# Pca

In [9]:
train_data = []
train_labels = []
for data, labels in train_loader_pca:
    train_data.append(data.view(data.size(0), -1))  
    train_labels.append(labels)
train_data = torch.cat(train_data, dim=0)  
train_labels = torch.cat(train_labels, dim=0)

train_data_np = train_data.numpy()

pca = PCADigitReducer(100)
train_data_reduced = pca.fit_transform(train_data_np)  

train_data_reconstructed_np = pca.inverse_transform(train_data_reduced) 
train_data_reconstructed = torch.tensor(train_data_reconstructed_np, dtype=torch.float32)

train_data_reconstructed = train_data_reconstructed.view(-1, 1, 28, 28)

train_data_reconstructed = (train_data_reconstructed - 0.0) / 1.0

batch_size_train = train_loader_pca.batch_size
train_dataset_pca = CustomTensorDataset(train_data_reconstructed, train_labels)
train_loader_reduced_pca = DataLoader(train_dataset_pca, batch_size=batch_size_train, shuffle=True)

# Autoencoder

In [10]:
# Autoencoder
latent_dim = 100  
autoencoder = Autoencoder(latent_dim=latent_dim)
auto_criterion = nn.MSELoss()
auto_optimizer = optim.Adam(autoencoder.parameters(), lr=1e-3)
auto_num_epochs = 5
for epoch in range(auto_num_epochs): 
    for images, _ in train_loader_auto:
        auto_optimizer.zero_grad()
        reconstructed = autoencoder(images)
        loss = auto_criterion(reconstructed, images)  
        loss.backward()
        auto_optimizer.step()
        
    print(f"Epoch [{epoch+1}/5], Loss: {loss.item()}")

Epoch [1/5], Loss: 0.020802266895771027
Epoch [2/5], Loss: 0.016157377511262894
Epoch [3/5], Loss: 0.013395553454756737
Epoch [4/5], Loss: 0.011990485712885857
Epoch [5/5], Loss: 0.01290703471750021


In [11]:
autoencoder.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
latent_features, labels = reduce_dimensions(train_loader_auto, autoencoder.encoder, device)
latent_features = latent_features.detach()

reconstructed_images = autoencoder.decoder(latent_features.to(device))  
reconstructed_images = reconstructed_images.view(-1, 1, 28, 28)  # Reshape to [batch_size, channels, height, width]

reconstructed_dataset = CustomTensorDataset(reconstructed_images.cpu(), labels)  
reduced_train_loader_auto = DataLoader(reconstructed_dataset, batch_size=batch_size_train, shuffle=True)

# Partition

## Classic Dataloaders

In [12]:
# classic
trainingset = fashion_mnist_train_loader.dataset
partitioned_data_classic = partition.balanced_dirichlet_partition(trainingset, partitions_number=4, alpha=0.5)

In [13]:
import cluster
cluster = cluster.Cluster(num_clusters=4)

targets = trainingset.targets
num_classes = len(set(targets)) 
clustered_data = cluster.apply_clustering(partitioned_data_classic, targets, num_classes)

partitioned_data_classic_clustered = clustered_data

AttributeError: module 'cluster' has no attribute 'Cluster'

In [14]:
# Normal loader classic
classic_client_loaders = [
    DataLoader(Subset(trainingset, indices), batch_size=batch_size_train, shuffle=True)
    for indices in partitioned_data_classic.values()
]

In [15]:
# Clustered loader classic
classic_client_loaders_clustered = [
    DataLoader(Subset(trainingset, indices), batch_size=batch_size_train, shuffle=True)
    for indices in partitioned_data_classic_clustered.values()
]

NameError: name 'partitioned_data_classic_clustered' is not defined

In [16]:
print(len(classic_client_loaders_clustered))

NameError: name 'classic_client_loaders_clustered' is not defined

## PCA Dataloaders

In [17]:
# pca 
trainingset_pca = train_loader_reduced_pca.dataset
partitioned_data_pca = partition.balanced_dirichlet_partition(trainingset_pca, partitions_number=4, alpha=0.5)

In [ ]:
import cluster
cluster = cluster.Cluster(num_clusters=4)

targets = trainingset_pca.targets
num_classes = len(set(targets)) 
clustered_data = cluster.apply_clustering(partitioned_data_pca, targets, num_classes)

partitioned_data_pca_clustered = clustered_data

In [18]:
# Normal loader pca
pca_client_loaders = [
    DataLoader(Subset(trainingset_pca, indices), batch_size=batch_size_train, shuffle=True)
    for indices in partitioned_data_pca.values()
]

In [ ]:
# Clustered loader pca
pca_client_loaders_clustered = [
    DataLoader(Subset(trainingset_pca, indices), batch_size=batch_size_train, shuffle=True)
    for indices in partitioned_data_pca_clustered.values()
]

In [ ]:
print(len(pca_client_loaders_clustered))

4


## Autoencoder Dataloaders

In [19]:
# Autoencoder
trainingset_auto = reduced_train_loader_auto.dataset
print(trainingset_auto.targets)
print(trainingset_auto.data.shape)
partitioned_data_auto = partition.balanced_dirichlet_partition(trainingset_auto, partitions_number=4, alpha=0.5)

tensor([6, 3, 2,  ..., 0, 6, 8])
torch.Size([60000, 1, 28, 28])


In [ ]:
import cluster
cluster = cluster.Cluster(num_clusters=4)

targets = trainingset_auto.targets
num_classes = len(set(targets)) 
clustered_data = cluster.apply_clustering(partitioned_data_auto, targets, num_classes)

partitioned_data_auto_clustered = clustered_data

In [20]:
# Normal loader autoencoder
auto_client_loaders = [
    DataLoader(Subset(trainingset_auto, indices), batch_size=batch_size_train, shuffle=True)
    for indices in partitioned_data_auto.values()
]

In [ ]:
# Clustered loader autoencoder
auto_client_loaders_clustered = [
    DataLoader(Subset(trainingset_auto, indices), batch_size=batch_size_train, shuffle=True)
    for indices in partitioned_data_auto_clustered.values()
]

In [ ]:
print(len(auto_client_loaders_clustered))

4


# Models

In [21]:
# Weak model
trial_model = classification_model()
trial_model_pca = classification_model()
trial_model_auto = classification_model()

global_model_pca = classification_model()
global_model_auto = classification_model()
global_model_classic = classification_model()

num_clients = 4
# classic models
local_models_classic = [copy.deepcopy(global_model_pca) for _ in range(num_clients)]
# pca models 
local_models_pca = [copy.deepcopy(global_model_pca) for _ in range(num_clients)]
# autoencodere models
local_model_autoencoder = [copy.deepcopy(global_model_pca) for _ in range(num_clients)]

In [22]:
# Stronger model

trial_model_strong = MultilayerPerceptron()
trial_model_pca_strong = MultilayerPerceptron()
trial_model_auto_strong = MultilayerPerceptron()

global_model_pca_strong = MultilayerPerceptron()
global_model_auto_strong = MultilayerPerceptron()
global_model_classic_strong = MultilayerPerceptron()

num_clients = 4
# classic models
local_models_classic_strong = [copy.deepcopy(global_model_classic_strong) for _ in range(num_clients)]
# pca models 
local_models_pca_strong = [copy.deepcopy(global_model_pca_strong) for _ in range(num_clients)]
# autoencodere models
local_model_autoencoder_strong = [copy.deepcopy(global_model_auto_strong) for _ in range(num_clients)]

# Training 

In [23]:
# test for errors

# Classic weak

optimizer = optim.SGD(trial_model.parameters(), lr=learning_rate,
                      momentum=momentum)

train_losses = []
train_counter = []

for epoch in range(1, n_epochs + 1):  
    train(epoch, trial_model, fashion_mnist_train_loader, optimizer, log_interval, train_losses, train_counter)

c:\Users\micha\Downloads\Federated-Dimensionality-Reduction\model2.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.296815
Train Epoch: 1 [1000/60000 (2%)]	Loss: 2.309201
Train Epoch: 1 [2000/60000 (3%)]	Loss: 2.297824
Train Epoch: 1 [3000/60000 (5%)]	Loss: 2.314433
Train Epoch: 1 [4000/60000 (7%)]	Loss: 2.300467
Train Epoch: 1 [5000/60000 (8%)]	Loss: 2.306074
Train Epoch: 1 [6000/60000 (10%)]	Loss: 2.287288
Train Epoch: 1 [7000/60000 (12%)]	Loss: 2.300402
Train Epoch: 1 [8000/60000 (13%)]	Loss: 2.280727
Train Epoch: 1 [9000/60000 (15%)]	Loss: 2.293508
Train Epoch: 1 [10000/60000 (17%)]	Loss: 2.284375
Train Epoch: 1 [11000/60000 (18%)]	Loss: 2.295316
Train Epoch: 1 [12000/60000 (20%)]	Loss: 2.295319
Train Epoch: 1 [13000/60000 (22%)]	Loss: 2.261042
Train Epoch: 1 [14000/60000 (23%)]	Loss: 2.271645
Train Epoch: 1 [15000/60000 (25%)]	Loss: 2.246674
Train Epoch: 1 [16000/60000 (27%)]	Loss: 2.228382
Train Epoch: 1 [17000/60000 (28%)]	Loss: 2.247286
Train Epoch: 1 [18000/60000 (30%)]	Loss: 2.206887
Train Epoch: 1 [19000/60000 (32%)]	Loss: 2.187974
Train Epoch: 1 [200

In [24]:
test_losses_classic_weak = []
test(trial_model,fashion_mnist_test_loader,test_losses_classic_weak)

c:\Users\micha\anaconda3\Lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 0.6449, Accuracy: 7524/10000 (75%)



In [25]:
# Pca weak

optimizer = optim.SGD(trial_model_pca.parameters(), lr=learning_rate,
                      momentum=momentum)

train_losses = []
train_counter = []

for epoch in range(1, n_epochs + 1):  
    train(epoch, trial_model_pca, train_loader_reduced_pca, optimizer, log_interval, train_losses, train_counter)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.304388
Train Epoch: 1 [1000/60000 (2%)]	Loss: 2.296096
Train Epoch: 1 [2000/60000 (3%)]	Loss: 2.309527
Train Epoch: 1 [3000/60000 (5%)]	Loss: 2.289254
Train Epoch: 1 [4000/60000 (7%)]	Loss: 2.297287
Train Epoch: 1 [5000/60000 (8%)]	Loss: 2.301597
Train Epoch: 1 [6000/60000 (10%)]	Loss: 2.288744
Train Epoch: 1 [7000/60000 (12%)]	Loss: 2.290226
Train Epoch: 1 [8000/60000 (13%)]	Loss: 2.285875
Train Epoch: 1 [9000/60000 (15%)]	Loss: 2.267900
Train Epoch: 1 [10000/60000 (17%)]	Loss: 2.246664
Train Epoch: 1 [11000/60000 (18%)]	Loss: 2.247575
Train Epoch: 1 [12000/60000 (20%)]	Loss: 2.284978
Train Epoch: 1 [13000/60000 (22%)]	Loss: 2.262689
Train Epoch: 1 [14000/60000 (23%)]	Loss: 2.243647
Train Epoch: 1 [15000/60000 (25%)]	Loss: 2.231950
Train Epoch: 1 [16000/60000 (27%)]	Loss: 2.240720
Train Epoch: 1 [17000/60000 (28%)]	Loss: 2.220482
Train Epoch: 1 [18000/60000 (30%)]	Loss: 2.182969
Train Epoch: 1 [19000/60000 (32%)]	Loss: 2.201971
Train Epoch: 1 [200

In [26]:
test_losses_classic_pca = []
test(trial_model_pca,train_loader_reduced_pca,test_losses_classic_pca)


Test set: Avg. loss: 0.5988, Accuracy: 45708/60000 (76%)



In [27]:
# Auto weak

optimizer = optim.SGD(trial_model_auto.parameters(), lr=learning_rate,
                      momentum=momentum)

train_losses = []
train_counter = []

for epoch in range(1, n_epochs + 1):  
    train(epoch, trial_model_auto, reduced_train_loader_auto, optimizer, log_interval, train_losses, train_counter)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.308264
Train Epoch: 1 [1000/60000 (2%)]	Loss: 2.285394
Train Epoch: 1 [2000/60000 (3%)]	Loss: 2.284465
Train Epoch: 1 [3000/60000 (5%)]	Loss: 2.290308
Train Epoch: 1 [4000/60000 (7%)]	Loss: 2.302227
Train Epoch: 1 [5000/60000 (8%)]	Loss: 2.291570
Train Epoch: 1 [6000/60000 (10%)]	Loss: 2.287755
Train Epoch: 1 [7000/60000 (12%)]	Loss: 2.298569
Train Epoch: 1 [8000/60000 (13%)]	Loss: 2.301424
Train Epoch: 1 [9000/60000 (15%)]	Loss: 2.273199
Train Epoch: 1 [10000/60000 (17%)]	Loss: 2.299524
Train Epoch: 1 [11000/60000 (18%)]	Loss: 2.281903
Train Epoch: 1 [12000/60000 (20%)]	Loss: 2.295039
Train Epoch: 1 [13000/60000 (22%)]	Loss: 2.285115
Train Epoch: 1 [14000/60000 (23%)]	Loss: 2.270465
Train Epoch: 1 [15000/60000 (25%)]	Loss: 2.268579
Train Epoch: 1 [16000/60000 (27%)]	Loss: 2.233048
Train Epoch: 1 [17000/60000 (28%)]	Loss: 2.211261
Train Epoch: 1 [18000/60000 (30%)]	Loss: 2.253889
Train Epoch: 1 [19000/60000 (32%)]	Loss: 2.237173
Train Epoch: 1 [200

In [28]:
test_losses_classic_auto = []
test(trial_model_auto,reduced_train_loader_auto,test_losses_classic_auto)


Test set: Avg. loss: 0.6560, Accuracy: 45046/60000 (75%)



In [29]:
# Classic strong

optimizer = optim.SGD(trial_model_strong.parameters(), lr=learning_rate,
                      momentum=momentum)

train_losses = []
train_counter = []

for epoch in range(1, n_epochs + 1):  
    train_fashion(epoch, trial_model_strong, fashion_mnist_train_loader, optimizer, log_interval, train_losses, train_counter)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.301711
Train Epoch: 1 [1000/60000 (2%)]	Loss: 2.291544
Train Epoch: 1 [2000/60000 (3%)]	Loss: 2.268291
Train Epoch: 1 [3000/60000 (5%)]	Loss: 2.246308
Train Epoch: 1 [4000/60000 (7%)]	Loss: 2.239659
Train Epoch: 1 [5000/60000 (8%)]	Loss: 2.197579
Train Epoch: 1 [6000/60000 (10%)]	Loss: 2.161548
Train Epoch: 1 [7000/60000 (12%)]	Loss: 2.148007
Train Epoch: 1 [8000/60000 (13%)]	Loss: 2.104607
Train Epoch: 1 [9000/60000 (15%)]	Loss: 2.079783
Train Epoch: 1 [10000/60000 (17%)]	Loss: 2.012955
Train Epoch: 1 [11000/60000 (18%)]	Loss: 1.899959
Train Epoch: 1 [12000/60000 (20%)]	Loss: 1.909051
Train Epoch: 1 [13000/60000 (22%)]	Loss: 1.773251
Train Epoch: 1 [14000/60000 (23%)]	Loss: 1.701815
Train Epoch: 1 [15000/60000 (25%)]	Loss: 1.654538
Train Epoch: 1 [16000/60000 (27%)]	Loss: 1.586564
Train Epoch: 1 [17000/60000 (28%)]	Loss: 1.477256
Train Epoch: 1 [18000/60000 (30%)]	Loss: 1.585987
Train Epoch: 1 [19000/60000 (32%)]	Loss: 1.433776
Train Epoch: 1 [200

In [30]:
test_losses_classic_strong = []
test_fashion(trial_model_strong,fashion_mnist_train_loader,test_losses_classic_strong)


Test set: Avg. loss: 0.4601, Accuracy: 50431/60000 (84%)



In [31]:
# Pca strong

optimizer = optim.SGD(trial_model_pca_strong.parameters(), lr=learning_rate,
                      momentum=momentum)

train_losses = []
train_counter = []

for epoch in range(1, n_epochs + 1):  
    train_fashion(epoch, trial_model_pca_strong, train_loader_reduced_pca, optimizer, log_interval, train_losses, train_counter)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.309561
Train Epoch: 1 [1000/60000 (2%)]	Loss: 2.288911
Train Epoch: 1 [2000/60000 (3%)]	Loss: 2.283791
Train Epoch: 1 [3000/60000 (5%)]	Loss: 2.270721
Train Epoch: 1 [4000/60000 (7%)]	Loss: 2.264411
Train Epoch: 1 [5000/60000 (8%)]	Loss: 2.247557
Train Epoch: 1 [6000/60000 (10%)]	Loss: 2.241054
Train Epoch: 1 [7000/60000 (12%)]	Loss: 2.195273
Train Epoch: 1 [8000/60000 (13%)]	Loss: 2.171674
Train Epoch: 1 [9000/60000 (15%)]	Loss: 2.157092
Train Epoch: 1 [10000/60000 (17%)]	Loss: 2.106555
Train Epoch: 1 [11000/60000 (18%)]	Loss: 2.106565
Train Epoch: 1 [12000/60000 (20%)]	Loss: 2.054374
Train Epoch: 1 [13000/60000 (22%)]	Loss: 2.003464
Train Epoch: 1 [14000/60000 (23%)]	Loss: 1.952736
Train Epoch: 1 [15000/60000 (25%)]	Loss: 1.850639
Train Epoch: 1 [16000/60000 (27%)]	Loss: 1.877702
Train Epoch: 1 [17000/60000 (28%)]	Loss: 1.804019
Train Epoch: 1 [18000/60000 (30%)]	Loss: 1.736062
Train Epoch: 1 [19000/60000 (32%)]	Loss: 1.647959
Train Epoch: 1 [200

In [32]:
test_losses_pca_strong = []
test_fashion(trial_model_pca_strong,train_loader_reduced_pca,test_losses_pca_strong)


Test set: Avg. loss: 0.4717, Accuracy: 50153/60000 (84%)



In [33]:
# Auto strong

optimizer = optim.SGD(trial_model_auto_strong.parameters(), lr=learning_rate,
                      momentum=momentum)

train_losses = []
train_counter = []

for epoch in range(1, n_epochs + 1):  
    train_fashion(epoch, trial_model_auto_strong, reduced_train_loader_auto, optimizer, log_interval, train_losses, train_counter)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.302374
Train Epoch: 1 [1000/60000 (2%)]	Loss: 2.284848
Train Epoch: 1 [2000/60000 (3%)]	Loss: 2.261323
Train Epoch: 1 [3000/60000 (5%)]	Loss: 2.245465
Train Epoch: 1 [4000/60000 (7%)]	Loss: 2.230203
Train Epoch: 1 [5000/60000 (8%)]	Loss: 2.213902
Train Epoch: 1 [6000/60000 (10%)]	Loss: 2.171041
Train Epoch: 1 [7000/60000 (12%)]	Loss: 2.125159
Train Epoch: 1 [8000/60000 (13%)]	Loss: 2.124658
Train Epoch: 1 [9000/60000 (15%)]	Loss: 2.057673
Train Epoch: 1 [10000/60000 (17%)]	Loss: 2.019439
Train Epoch: 1 [11000/60000 (18%)]	Loss: 1.951476
Train Epoch: 1 [12000/60000 (20%)]	Loss: 1.854320
Train Epoch: 1 [13000/60000 (22%)]	Loss: 1.767654
Train Epoch: 1 [14000/60000 (23%)]	Loss: 1.743414
Train Epoch: 1 [15000/60000 (25%)]	Loss: 1.691292
Train Epoch: 1 [16000/60000 (27%)]	Loss: 1.570073
Train Epoch: 1 [17000/60000 (28%)]	Loss: 1.538807
Train Epoch: 1 [18000/60000 (30%)]	Loss: 1.508415
Train Epoch: 1 [19000/60000 (32%)]	Loss: 1.406113
Train Epoch: 1 [200

In [34]:
test_losses_auto_strong = []
test_fashion(trial_model_auto_strong,reduced_train_loader_auto,test_losses_auto_strong)


Test set: Avg. loss: 0.5105, Accuracy: 49113/60000 (82%)



## Weak Model

In [35]:
# Classic

rounds_classic = 4

for round_idx in range(rounds_classic):
    
    print(f"Round {round_idx + 1}/{rounds_classic}")

    local_weights_classic = []
    for client_idx, client_model in enumerate(local_models_classic):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)

        train_losses = []
        train_counter = []

        for epoch in range(1, n_epochs + 1):  
            train(epoch, client_model, classic_client_loaders[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_classic.append(client_weights)
        
    print(f"after training{local_models_classic}")
    global_weights_classic = federated_averaging(local_weights_classic)
    print(f"after fedaveraging{local_models_classic}")

    distribute_global_model(global_weights_classic,local_models_classic,single=False)

    distribute_global_model(global_weights_classic,global_model_classic,single=True)
    test_losses = []
    test(global_model_classic,fashion_mnist_test_loader,test_losses)

Round 1/4
Training client 1
Train Epoch: 1 [0/23974 (0%)]	Loss: 2.323832
Train Epoch: 1 [1000/23974 (4%)]	Loss: 2.281301
Train Epoch: 1 [2000/23974 (8%)]	Loss: 2.271091
Train Epoch: 1 [3000/23974 (12%)]	Loss: 2.182160
Train Epoch: 1 [4000/23974 (17%)]	Loss: 2.235679
Train Epoch: 1 [5000/23974 (21%)]	Loss: 2.147833
Train Epoch: 1 [6000/23974 (25%)]	Loss: 2.174901
Train Epoch: 1 [7000/23974 (29%)]	Loss: 2.058657
Train Epoch: 1 [8000/23974 (33%)]	Loss: 2.102009
Train Epoch: 1 [9000/23974 (38%)]	Loss: 2.113300
Train Epoch: 1 [10000/23974 (42%)]	Loss: 2.185816
Train Epoch: 1 [11000/23974 (46%)]	Loss: 2.054514
Train Epoch: 1 [12000/23974 (50%)]	Loss: 2.001182
Train Epoch: 1 [13000/23974 (54%)]	Loss: 2.017333
Train Epoch: 1 [14000/23974 (58%)]	Loss: 2.109258
Train Epoch: 1 [15000/23974 (62%)]	Loss: 1.963540
Train Epoch: 1 [16000/23974 (67%)]	Loss: 1.940617
Train Epoch: 1 [17000/23974 (71%)]	Loss: 1.923892
Train Epoch: 1 [18000/23974 (75%)]	Loss: 1.827722
Train Epoch: 1 [19000/23974 (79%)]	Los

In [36]:
# PCA

rounds_pca = 4

for round_idx in range(rounds_pca):
    
    print(f"Round {round_idx + 1}/{rounds_pca}")

    local_weights_pca = []
    for client_idx, client_model in enumerate(local_models_pca):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)

        train_losses = []
        train_counter = []


        for epoch in range(1, n_epochs + 1):  
            train(epoch, client_model, pca_client_loaders[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_pca.append(client_weights)
        
    print(f"after training{local_models_pca}")
    global_weights_pca = federated_averaging(local_weights_pca)
    print(f"after fedaveraging{local_models_pca}")

    distribute_global_model(global_weights_pca,local_models_pca,single=False)

    distribute_global_model(global_weights_pca,global_model_pca,single=True)
    test_losses = []
    test(global_model_pca,test_loader_pca,test_losses)

Round 1/4
Training client 1
Train Epoch: 1 [0/23974 (0%)]	Loss: 2.303671
Train Epoch: 1 [1000/23974 (4%)]	Loss: 2.286044
Train Epoch: 1 [2000/23974 (8%)]	Loss: 2.243041
Train Epoch: 1 [3000/23974 (12%)]	Loss: 2.259393
Train Epoch: 1 [4000/23974 (17%)]	Loss: 2.175682
Train Epoch: 1 [5000/23974 (21%)]	Loss: 2.176730
Train Epoch: 1 [6000/23974 (25%)]	Loss: 2.139965
Train Epoch: 1 [7000/23974 (29%)]	Loss: 2.067148
Train Epoch: 1 [8000/23974 (33%)]	Loss: 2.114075
Train Epoch: 1 [9000/23974 (38%)]	Loss: 2.075353
Train Epoch: 1 [10000/23974 (42%)]	Loss: 1.975879
Train Epoch: 1 [11000/23974 (46%)]	Loss: 1.989539
Train Epoch: 1 [12000/23974 (50%)]	Loss: 2.013371
Train Epoch: 1 [13000/23974 (54%)]	Loss: 2.046387
Train Epoch: 1 [14000/23974 (58%)]	Loss: 2.147349
Train Epoch: 1 [15000/23974 (62%)]	Loss: 1.977089
Train Epoch: 1 [16000/23974 (67%)]	Loss: 1.950266
Train Epoch: 1 [17000/23974 (71%)]	Loss: 1.933297
Train Epoch: 1 [18000/23974 (75%)]	Loss: 1.928498
Train Epoch: 1 [19000/23974 (79%)]	Los

In [37]:
# Autoencoder

rounds_auto = 4

for round_idx in range(rounds_auto):
    print(f"Round {round_idx + 1}/{rounds_auto}")

    local_weights_auto = []
    for client_idx, client_model in enumerate(local_model_autoencoder):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)
        
        train_losses = []
        train_counter = []

        for epoch in range(1, n_epochs + 1):  
            train(epoch, client_model, auto_client_loaders[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_auto.append(client_weights)
        
    global_weights_auto = federated_averaging(local_weights_auto)

    distribute_global_model(global_weights_auto,local_model_autoencoder,single=False)

    distribute_global_model(global_weights_auto, global_model_auto,single=True)
    test_losses = []
    test(global_model_auto,test_loader_auto,test_losses)

Round 1/4
Training client 1
Train Epoch: 1 [0/23974 (0%)]	Loss: 2.329380
Train Epoch: 1 [1000/23974 (4%)]	Loss: 2.274496
Train Epoch: 1 [2000/23974 (8%)]	Loss: 2.257426
Train Epoch: 1 [3000/23974 (12%)]	Loss: 2.222621
Train Epoch: 1 [4000/23974 (17%)]	Loss: 2.239375
Train Epoch: 1 [5000/23974 (21%)]	Loss: 2.182172
Train Epoch: 1 [6000/23974 (25%)]	Loss: 2.222055
Train Epoch: 1 [7000/23974 (29%)]	Loss: 2.227219
Train Epoch: 1 [8000/23974 (33%)]	Loss: 2.182452
Train Epoch: 1 [9000/23974 (38%)]	Loss: 2.136250
Train Epoch: 1 [10000/23974 (42%)]	Loss: 2.107624
Train Epoch: 1 [11000/23974 (46%)]	Loss: 2.140660
Train Epoch: 1 [12000/23974 (50%)]	Loss: 2.029768
Train Epoch: 1 [13000/23974 (54%)]	Loss: 2.051964
Train Epoch: 1 [14000/23974 (58%)]	Loss: 2.053383
Train Epoch: 1 [15000/23974 (62%)]	Loss: 1.989198
Train Epoch: 1 [16000/23974 (67%)]	Loss: 1.992311
Train Epoch: 1 [17000/23974 (71%)]	Loss: 2.016902
Train Epoch: 1 [18000/23974 (75%)]	Loss: 1.951637
Train Epoch: 1 [19000/23974 (79%)]	Los

## Strong model

In [38]:
# Classic

rounds_classic = 4

for round_idx in range(rounds_classic):
    
    print(f"Round {round_idx + 1}/{rounds_classic}")

    local_weights_classic = []
    for client_idx, client_model in enumerate(local_models_classic_strong):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)

        train_losses = []
        train_counter = []

        for epoch in range(1, n_epochs + 1):  
            train_fashion(epoch, client_model, classic_client_loaders[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_classic.append(client_weights)
        

    global_weights_classic = federated_averaging(local_weights_classic)


    distribute_global_model(global_weights_classic,local_models_classic_strong,single=False)

    distribute_global_model(global_weights_classic,global_model_classic_strong,single=True)
    test_losses = []
    test_fashion(global_model_classic_strong,fashion_mnist_test_loader,test_losses)

Round 1/4
Training client 1
Train Epoch: 1 [0/23974 (0%)]	Loss: 2.301013
Train Epoch: 1 [1000/23974 (4%)]	Loss: 2.245649
Train Epoch: 1 [2000/23974 (8%)]	Loss: 2.194252
Train Epoch: 1 [3000/23974 (12%)]	Loss: 2.080554
Train Epoch: 1 [4000/23974 (17%)]	Loss: 2.048407
Train Epoch: 1 [5000/23974 (21%)]	Loss: 2.009072
Train Epoch: 1 [6000/23974 (25%)]	Loss: 1.865758
Train Epoch: 1 [7000/23974 (29%)]	Loss: 1.811684
Train Epoch: 1 [8000/23974 (33%)]	Loss: 1.818650
Train Epoch: 1 [9000/23974 (38%)]	Loss: 1.797668
Train Epoch: 1 [10000/23974 (42%)]	Loss: 1.617993
Train Epoch: 1 [11000/23974 (46%)]	Loss: 1.702526
Train Epoch: 1 [12000/23974 (50%)]	Loss: 1.632206
Train Epoch: 1 [13000/23974 (54%)]	Loss: 1.589964
Train Epoch: 1 [14000/23974 (58%)]	Loss: 1.319359
Train Epoch: 1 [15000/23974 (62%)]	Loss: 1.321224
Train Epoch: 1 [16000/23974 (67%)]	Loss: 1.172715
Train Epoch: 1 [17000/23974 (71%)]	Loss: 1.280549
Train Epoch: 1 [18000/23974 (75%)]	Loss: 1.372143
Train Epoch: 1 [19000/23974 (79%)]	Los

In [45]:
# PCA

rounds_pca = 8

for round_idx in range(rounds_pca):
    
    print(f"Round {round_idx + 1}/{rounds_pca}")

    local_weights_pca = []
    for client_idx, client_model in enumerate(local_models_pca_strong):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)

        train_losses = []
        train_counter = []


        for epoch in range(1, n_epochs + 1):  
            train_fashion(epoch, client_model, pca_client_loaders[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_pca.append(client_weights)
        

    global_weights_pca = federated_averaging(local_weights_pca)

    distribute_global_model(global_weights_pca,local_models_pca_strong,single=False)

    distribute_global_model(global_weights_pca,global_model_pca_strong,single=True)
    test_losses = []
    test_fashion(global_model_pca_strong,test_loader_pca,test_losses)

Round 1/8
Training client 1
Train Epoch: 1 [0/23974 (0%)]	Loss: 0.210290
Train Epoch: 1 [1000/23974 (4%)]	Loss: 0.258347
Train Epoch: 1 [2000/23974 (8%)]	Loss: 0.187031
Train Epoch: 1 [3000/23974 (12%)]	Loss: 0.185966
Train Epoch: 1 [4000/23974 (17%)]	Loss: 0.186264
Train Epoch: 1 [5000/23974 (21%)]	Loss: 0.227356
Train Epoch: 1 [6000/23974 (25%)]	Loss: 0.299231
Train Epoch: 1 [7000/23974 (29%)]	Loss: 0.240308
Train Epoch: 1 [8000/23974 (33%)]	Loss: 0.218805
Train Epoch: 1 [9000/23974 (38%)]	Loss: 0.268353
Train Epoch: 1 [10000/23974 (42%)]	Loss: 0.132978
Train Epoch: 1 [11000/23974 (46%)]	Loss: 0.191670
Train Epoch: 1 [12000/23974 (50%)]	Loss: 0.250270
Train Epoch: 1 [13000/23974 (54%)]	Loss: 0.250428
Train Epoch: 1 [14000/23974 (58%)]	Loss: 0.215167
Train Epoch: 1 [15000/23974 (62%)]	Loss: 0.111443
Train Epoch: 1 [16000/23974 (67%)]	Loss: 0.385846
Train Epoch: 1 [17000/23974 (71%)]	Loss: 0.210720
Train Epoch: 1 [18000/23974 (75%)]	Loss: 0.239725
Train Epoch: 1 [19000/23974 (79%)]	Los

In [40]:
# Autoencoder

rounds_auto = 4

for round_idx in range(rounds_auto):
    print(f"Round {round_idx + 1}/{rounds_auto}")

    local_weights_auto = []
    for client_idx, client_model in enumerate(local_model_autoencoder_strong):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)
        
        train_losses = []
        train_counter = []

        for epoch in range(1, n_epochs + 1):  
            train_fashion(epoch, client_model, auto_client_loaders[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_auto.append(client_weights)
        
    global_weights_auto = federated_averaging(local_weights_auto)

    distribute_global_model(global_weights_auto,local_model_autoencoder_strong,single=False)

    distribute_global_model(global_weights_auto, global_model_auto_strong,single=True)
    test_losses = []
    test_fashion(global_model_auto_strong,test_loader_auto,test_losses)

Round 1/4
Training client 1
Train Epoch: 1 [0/23974 (0%)]	Loss: 2.306906
Train Epoch: 1 [1000/23974 (4%)]	Loss: 2.274323
Train Epoch: 1 [2000/23974 (8%)]	Loss: 2.234305
Train Epoch: 1 [3000/23974 (12%)]	Loss: 2.201439
Train Epoch: 1 [4000/23974 (17%)]	Loss: 2.098862
Train Epoch: 1 [5000/23974 (21%)]	Loss: 2.009043
Train Epoch: 1 [6000/23974 (25%)]	Loss: 2.010259
Train Epoch: 1 [7000/23974 (29%)]	Loss: 1.849209
Train Epoch: 1 [8000/23974 (33%)]	Loss: 1.778690
Train Epoch: 1 [9000/23974 (38%)]	Loss: 1.684687
Train Epoch: 1 [10000/23974 (42%)]	Loss: 1.726052
Train Epoch: 1 [11000/23974 (46%)]	Loss: 1.623490
Train Epoch: 1 [12000/23974 (50%)]	Loss: 1.615811
Train Epoch: 1 [13000/23974 (54%)]	Loss: 1.517358
Train Epoch: 1 [14000/23974 (58%)]	Loss: 1.479888
Train Epoch: 1 [15000/23974 (62%)]	Loss: 1.442060
Train Epoch: 1 [16000/23974 (67%)]	Loss: 1.402424
Train Epoch: 1 [17000/23974 (71%)]	Loss: 1.228058
Train Epoch: 1 [18000/23974 (75%)]	Loss: 1.270602
Train Epoch: 1 [19000/23974 (79%)]	Los

# Cluster

In [40]:
# Classic

rounds_classic = 4

for round_idx in range(rounds_classic):
    
    print(f"Round {round_idx + 1}/{rounds_classic}")

    local_weights_classic = []
    for client_idx, client_model in enumerate(local_models_classic):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)

        train_losses = []
        train_counter = []

        for epoch in range(1, n_epochs + 1):  
            train(epoch, client_model, classic_client_loaders_clustered[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_classic.append(client_weights)
        
    print(f"after training{local_models_classic}")
    global_weights_classic = federated_averaging(local_weights_classic)
    print(f"after fedaveraging{local_models_classic}")

    distribute_global_model(global_weights_classic,local_models_classic,single=False)

    distribute_global_model(global_weights_classic,global_model_classic,single=True)
    test_losses = []
    test(global_model_classic,fashion_mnist_test_loader,test_losses)

Round 1/4
Training client 1
Train Epoch: 1 [0/23974 (0%)]	Loss: 0.456145


c:\Users\micha\Downloads\Federated-Dimensionality-Reduction\model2.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 [1000/23974 (4%)]	Loss: 0.603035
Train Epoch: 1 [2000/23974 (8%)]	Loss: 0.682673
Train Epoch: 1 [3000/23974 (12%)]	Loss: 0.568576
Train Epoch: 1 [4000/23974 (17%)]	Loss: 0.827289
Train Epoch: 1 [5000/23974 (21%)]	Loss: 0.507216
Train Epoch: 1 [6000/23974 (25%)]	Loss: 0.541411
Train Epoch: 1 [7000/23974 (29%)]	Loss: 0.702418
Train Epoch: 1 [8000/23974 (33%)]	Loss: 0.641412
Train Epoch: 1 [9000/23974 (38%)]	Loss: 0.717685
Train Epoch: 1 [10000/23974 (42%)]	Loss: 0.622384
Train Epoch: 1 [11000/23974 (46%)]	Loss: 0.684873
Train Epoch: 1 [12000/23974 (50%)]	Loss: 0.523795
Train Epoch: 1 [13000/23974 (54%)]	Loss: 0.577843
Train Epoch: 1 [14000/23974 (58%)]	Loss: 0.649042
Train Epoch: 1 [15000/23974 (62%)]	Loss: 0.733746
Train Epoch: 1 [16000/23974 (67%)]	Loss: 0.646495
Train Epoch: 1 [17000/23974 (71%)]	Loss: 0.694190
Train Epoch: 1 [18000/23974 (75%)]	Loss: 0.658335
Train Epoch: 1 [19000/23974 (79%)]	Loss: 0.756742
Train Epoch: 1 [20000/23974 (83%)]	Loss: 0.520052
Train Epoch

c:\Users\micha\anaconda3\Lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 0.7751, Accuracy: 6998/10000 (70%)

Round 2/4
Training client 1
Train Epoch: 1 [0/23974 (0%)]	Loss: 1.021651
Train Epoch: 1 [1000/23974 (4%)]	Loss: 0.955496
Train Epoch: 1 [2000/23974 (8%)]	Loss: 0.768518
Train Epoch: 1 [3000/23974 (12%)]	Loss: 0.864623
Train Epoch: 1 [4000/23974 (17%)]	Loss: 0.724258
Train Epoch: 1 [5000/23974 (21%)]	Loss: 0.808818
Train Epoch: 1 [6000/23974 (25%)]	Loss: 0.859125
Train Epoch: 1 [7000/23974 (29%)]	Loss: 0.798802
Train Epoch: 1 [8000/23974 (33%)]	Loss: 0.700376
Train Epoch: 1 [9000/23974 (38%)]	Loss: 0.625857
Train Epoch: 1 [10000/23974 (42%)]	Loss: 0.707647
Train Epoch: 1 [11000/23974 (46%)]	Loss: 0.512070
Train Epoch: 1 [12000/23974 (50%)]	Loss: 0.726743
Train Epoch: 1 [13000/23974 (54%)]	Loss: 0.627127
Train Epoch: 1 [14000/23974 (58%)]	Loss: 0.586230
Train Epoch: 1 [15000/23974 (62%)]	Loss: 0.638155
Train Epoch: 1 [16000/23974 (67%)]	Loss: 0.755404
Train Epoch: 1 [17000/23974 (71%)]	Loss: 0.605010
Train Epoch: 1 [18000/23974 (7

In [41]:
# PCA

rounds_pca = 4

for round_idx in range(rounds_pca):
    
    print(f"Round {round_idx + 1}/{rounds_pca}")

    local_weights_pca = []
    for client_idx, client_model in enumerate(local_models_pca):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)

        train_losses = []
        train_counter = []


        for epoch in range(1, n_epochs + 1):  
            train(epoch, client_model, pca_client_loaders_clustered[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_pca.append(client_weights)
        
    print(f"after training{local_models_pca}")
    global_weights_pca = federated_averaging(local_weights_pca)
    print(f"after fedaveraging{local_models_pca}")

    distribute_global_model(global_weights_pca,local_models_pca,single=False)

    distribute_global_model(global_weights_pca,global_model_pca,single=True)
    test_losses = []
    test(global_model_pca,test_loader_pca,test_losses)

Round 1/4
Training client 1
Train Epoch: 1 [0/23974 (0%)]	Loss: 2.485735
Train Epoch: 1 [1000/23974 (4%)]	Loss: 2.201648
Train Epoch: 1 [2000/23974 (8%)]	Loss: 2.083010
Train Epoch: 1 [3000/23974 (12%)]	Loss: 1.977413
Train Epoch: 1 [4000/23974 (17%)]	Loss: 1.920428
Train Epoch: 1 [5000/23974 (21%)]	Loss: 1.600373
Train Epoch: 1 [6000/23974 (25%)]	Loss: 1.571871
Train Epoch: 1 [7000/23974 (29%)]	Loss: 1.504004
Train Epoch: 1 [8000/23974 (33%)]	Loss: 1.142571
Train Epoch: 1 [9000/23974 (38%)]	Loss: 1.258850
Train Epoch: 1 [10000/23974 (42%)]	Loss: 1.290202
Train Epoch: 1 [11000/23974 (46%)]	Loss: 1.225020
Train Epoch: 1 [12000/23974 (50%)]	Loss: 1.164176
Train Epoch: 1 [13000/23974 (54%)]	Loss: 1.057271
Train Epoch: 1 [14000/23974 (58%)]	Loss: 1.046755
Train Epoch: 1 [15000/23974 (62%)]	Loss: 1.053828
Train Epoch: 1 [16000/23974 (67%)]	Loss: 0.865568
Train Epoch: 1 [17000/23974 (71%)]	Loss: 0.989750
Train Epoch: 1 [18000/23974 (75%)]	Loss: 0.820729
Train Epoch: 1 [19000/23974 (79%)]	Los

In [42]:
# Autoencoder

rounds_auto = 4

for round_idx in range(rounds_auto):
    print(f"Round {round_idx + 1}/{rounds_auto}")

    local_weights_auto = []
    for client_idx, client_model in enumerate(local_model_autoencoder):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)
        
        train_losses = []
        train_counter = []

        for epoch in range(1, n_epochs + 1):  
            train(epoch, client_model, auto_client_loaders_clustered[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_auto.append(client_weights)
        
    global_weights_auto = federated_averaging(local_weights_auto)

    distribute_global_model(global_weights_auto,local_model_autoencoder,single=False)

    distribute_global_model(global_weights_auto, global_model_auto,single=True)
    test_losses = []
    test(global_model_auto,test_loader_auto,test_losses)

Round 1/4
Training client 1
Train Epoch: 1 [0/23974 (0%)]	Loss: 2.343646
Train Epoch: 1 [1000/23974 (4%)]	Loss: 2.270311
Train Epoch: 1 [2000/23974 (8%)]	Loss: 2.251366
Train Epoch: 1 [3000/23974 (12%)]	Loss: 2.248857
Train Epoch: 1 [4000/23974 (17%)]	Loss: 2.177566
Train Epoch: 1 [5000/23974 (21%)]	Loss: 2.074116
Train Epoch: 1 [6000/23974 (25%)]	Loss: 2.181942
Train Epoch: 1 [7000/23974 (29%)]	Loss: 2.038073
Train Epoch: 1 [8000/23974 (33%)]	Loss: 2.171406
Train Epoch: 1 [9000/23974 (38%)]	Loss: 2.084585
Train Epoch: 1 [10000/23974 (42%)]	Loss: 2.109407
Train Epoch: 1 [11000/23974 (46%)]	Loss: 2.013240
Train Epoch: 1 [12000/23974 (50%)]	Loss: 2.116208
Train Epoch: 1 [13000/23974 (54%)]	Loss: 2.112153
Train Epoch: 1 [14000/23974 (58%)]	Loss: 2.049365
Train Epoch: 1 [15000/23974 (62%)]	Loss: 1.981739
Train Epoch: 1 [16000/23974 (67%)]	Loss: 1.975205
Train Epoch: 1 [17000/23974 (71%)]	Loss: 1.898549
Train Epoch: 1 [18000/23974 (75%)]	Loss: 1.813389
Train Epoch: 1 [19000/23974 (79%)]	Los

# Strong Model Cluster

In [43]:
# Classic

rounds_classic = 4

for round_idx in range(rounds_classic):
    
    print(f"Round {round_idx + 1}/{rounds_classic}")

    local_weights_classic = []
    for client_idx, client_model in enumerate(local_models_classic_strong):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)

        train_losses = []
        train_counter = []

        for epoch in range(1, n_epochs + 1):  
            train_fashion(epoch, client_model, classic_client_loaders_clustered[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_classic.append(client_weights)
        

    global_weights_classic = federated_averaging(local_weights_classic)


    distribute_global_model(global_weights_classic,local_models_classic_strong,single=False)

    distribute_global_model(global_weights_classic,global_model_classic_strong,single=True)
    test_losses = []
    test_fashion(global_model_classic_strong,fashion_mnist_test_loader,test_losses)

Round 1/4
Training client 1
Train Epoch: 1 [0/23974 (0%)]	Loss: 0.522776
Train Epoch: 1 [1000/23974 (4%)]	Loss: 0.337348
Train Epoch: 1 [2000/23974 (8%)]	Loss: 0.322143
Train Epoch: 1 [3000/23974 (12%)]	Loss: 0.294087
Train Epoch: 1 [4000/23974 (17%)]	Loss: 0.348672
Train Epoch: 1 [5000/23974 (21%)]	Loss: 0.224583
Train Epoch: 1 [6000/23974 (25%)]	Loss: 0.555698
Train Epoch: 1 [7000/23974 (29%)]	Loss: 0.282498
Train Epoch: 1 [8000/23974 (33%)]	Loss: 0.352552
Train Epoch: 1 [9000/23974 (38%)]	Loss: 0.379134
Train Epoch: 1 [10000/23974 (42%)]	Loss: 0.220214
Train Epoch: 1 [11000/23974 (46%)]	Loss: 0.322660
Train Epoch: 1 [12000/23974 (50%)]	Loss: 0.367432
Train Epoch: 1 [13000/23974 (54%)]	Loss: 0.326016
Train Epoch: 1 [14000/23974 (58%)]	Loss: 0.234246
Train Epoch: 1 [15000/23974 (62%)]	Loss: 0.412466
Train Epoch: 1 [16000/23974 (67%)]	Loss: 0.486033
Train Epoch: 1 [17000/23974 (71%)]	Loss: 0.202997
Train Epoch: 1 [18000/23974 (75%)]	Loss: 0.205094
Train Epoch: 1 [19000/23974 (79%)]	Los

c:\Users\micha\anaconda3\Lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 0.4239, Accuracy: 8457/10000 (85%)

Round 2/4
Training client 1
Train Epoch: 1 [0/23974 (0%)]	Loss: 0.376190
Train Epoch: 1 [1000/23974 (4%)]	Loss: 0.272269
Train Epoch: 1 [2000/23974 (8%)]	Loss: 0.316284
Train Epoch: 1 [3000/23974 (12%)]	Loss: 0.257268
Train Epoch: 1 [4000/23974 (17%)]	Loss: 0.316200
Train Epoch: 1 [5000/23974 (21%)]	Loss: 0.396427
Train Epoch: 1 [6000/23974 (25%)]	Loss: 0.316117
Train Epoch: 1 [7000/23974 (29%)]	Loss: 0.288723
Train Epoch: 1 [8000/23974 (33%)]	Loss: 0.283512
Train Epoch: 1 [9000/23974 (38%)]	Loss: 0.261743
Train Epoch: 1 [10000/23974 (42%)]	Loss: 0.254122
Train Epoch: 1 [11000/23974 (46%)]	Loss: 0.469816
Train Epoch: 1 [12000/23974 (50%)]	Loss: 0.168087
Train Epoch: 1 [13000/23974 (54%)]	Loss: 0.241160
Train Epoch: 1 [14000/23974 (58%)]	Loss: 0.329779
Train Epoch: 1 [15000/23974 (62%)]	Loss: 0.174850
Train Epoch: 1 [16000/23974 (67%)]	Loss: 0.234565
Train Epoch: 1 [17000/23974 (71%)]	Loss: 0.217776
Train Epoch: 1 [18000/23974 (7

In [44]:
# PCA

rounds_pca = 4

for round_idx in range(rounds_pca):
    
    print(f"Round {round_idx + 1}/{rounds_pca}")

    local_weights_pca = []
    for client_idx, client_model in enumerate(local_models_pca_strong):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)

        train_losses = []
        train_counter = []


        for epoch in range(1, n_epochs + 1):  
            train_fashion(epoch, client_model, pca_client_loaders_clustered[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_pca.append(client_weights)
        

    global_weights_pca = federated_averaging(local_weights_pca)

    distribute_global_model(global_weights_pca,local_models_pca_strong,single=False)

    distribute_global_model(global_weights_pca,global_model_pca_strong,single=True)
    test_losses = []
    test_fashion(global_model_pca_strong,test_loader_pca,test_losses)

Round 1/4
Training client 1
Train Epoch: 1 [0/23974 (0%)]	Loss: 0.491333
Train Epoch: 1 [1000/23974 (4%)]	Loss: 0.227463
Train Epoch: 1 [2000/23974 (8%)]	Loss: 0.233770
Train Epoch: 1 [3000/23974 (12%)]	Loss: 0.247685
Train Epoch: 1 [4000/23974 (17%)]	Loss: 0.494265
Train Epoch: 1 [5000/23974 (21%)]	Loss: 0.413155
Train Epoch: 1 [6000/23974 (25%)]	Loss: 0.333106
Train Epoch: 1 [7000/23974 (29%)]	Loss: 0.341256
Train Epoch: 1 [8000/23974 (33%)]	Loss: 0.192805
Train Epoch: 1 [9000/23974 (38%)]	Loss: 0.269251
Train Epoch: 1 [10000/23974 (42%)]	Loss: 0.257719
Train Epoch: 1 [11000/23974 (46%)]	Loss: 0.243606
Train Epoch: 1 [12000/23974 (50%)]	Loss: 0.263083
Train Epoch: 1 [13000/23974 (54%)]	Loss: 0.304439
Train Epoch: 1 [14000/23974 (58%)]	Loss: 0.210762
Train Epoch: 1 [15000/23974 (62%)]	Loss: 0.156424
Train Epoch: 1 [16000/23974 (67%)]	Loss: 0.200947
Train Epoch: 1 [17000/23974 (71%)]	Loss: 0.284574
Train Epoch: 1 [18000/23974 (75%)]	Loss: 0.298043
Train Epoch: 1 [19000/23974 (79%)]	Los

In [45]:
# Autoencoder

rounds_auto = 4

for round_idx in range(rounds_auto):
    print(f"Round {round_idx + 1}/{rounds_auto}")

    local_weights_auto = []
    for client_idx, client_model in enumerate(local_model_autoencoder_strong):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)
        
        train_losses = []
        train_counter = []

        for epoch in range(1, n_epochs + 1):  
            train_fashion(epoch, client_model, auto_client_loaders_clustered[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_auto.append(client_weights)
        
    global_weights_auto = federated_averaging(local_weights_auto)

    distribute_global_model(global_weights_auto,local_model_autoencoder_strong,single=False)

    distribute_global_model(global_weights_auto, global_model_auto_strong,single=True)
    test_losses = []
    test_fashion(global_model_auto_strong,test_loader_auto,test_losses)

Round 1/4
Training client 1
Train Epoch: 1 [0/23974 (0%)]	Loss: 0.493551
Train Epoch: 1 [1000/23974 (4%)]	Loss: 0.583331
Train Epoch: 1 [2000/23974 (8%)]	Loss: 0.462129
Train Epoch: 1 [3000/23974 (12%)]	Loss: 0.420153
Train Epoch: 1 [4000/23974 (17%)]	Loss: 0.454641
Train Epoch: 1 [5000/23974 (21%)]	Loss: 0.488483
Train Epoch: 1 [6000/23974 (25%)]	Loss: 0.471783
Train Epoch: 1 [7000/23974 (29%)]	Loss: 0.494689
Train Epoch: 1 [8000/23974 (33%)]	Loss: 0.386161
Train Epoch: 1 [9000/23974 (38%)]	Loss: 0.417625
Train Epoch: 1 [10000/23974 (42%)]	Loss: 0.548303
Train Epoch: 1 [11000/23974 (46%)]	Loss: 0.318957
Train Epoch: 1 [12000/23974 (50%)]	Loss: 0.411810
Train Epoch: 1 [13000/23974 (54%)]	Loss: 0.556645
Train Epoch: 1 [14000/23974 (58%)]	Loss: 0.518286
Train Epoch: 1 [15000/23974 (62%)]	Loss: 0.568789
Train Epoch: 1 [16000/23974 (67%)]	Loss: 0.590449
Train Epoch: 1 [17000/23974 (71%)]	Loss: 0.384189
Train Epoch: 1 [18000/23974 (75%)]	Loss: 0.298780
Train Epoch: 1 [19000/23974 (79%)]	Los